<a href="https://colab.research.google.com/github/hossain-mdismail/Cryptography_Lab_3/blob/main/Cryptography_Lab_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
pip install cryptography

In [5]:
# All necessary imports for the entire lab
import sys
import os
import time
import base64
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend

In [6]:
# Imports from our provided script
from AISC_03 import (
    encodeText, decodeText,
    generate_prime_candidate, is_prime, egcd,
    encryptAESCTR, decryptAESCTR,
    pDH, gDH, qDH
)

print("All libraries and functions imported successfully.")

All libraries and functions imported successfully.


#### PART 1: RSA WARM-UP (ENCRYPT FOR INSTRUCTOR)

In [13]:

# --- TODO:---

# 1. Getting the instructor's public key (N, e) from the Google Sheet

INSTRUCTOR_N = 84679728665601623534964724907925515929006540043189517481604602443064696359792213876789134073333039587280256381629121330309212710075084136072434602842735932397920567074126295940793758270599694063151970957022891161811181118800418280560581409959172714364916927401331291518944188508877716559336163266657183044021

INSTRUCTOR_E = 65537

# 2. Writing a short message to the instructor.
#    Keeping it short so it encodes to a single integer
MY_MESSAGE = "Hello Instructor! This is Md Ismail Hossain."


def main():
    print("Starting Lab 03 - Part 1: RSA Warm-up")

    # --- 1. Validation ---
    if INSTRUCTOR_N is None or INSTRUCTOR_E is None:
        print("\n" + "="*70)
        print(" ERROR: Please edit 'python file' and fill in")
        print("        'INSTRUCTOR_N' and 'INSTRUCTOR_E' with the values")
        print("        from the Google Sheet.")
        print("="*70)
        sys.exit(1)

    if "[Name]" in MY_MESSAGE:
        print("\n" + "="*70)
        print(" ERROR: Please edit 'lab03_part1.py' and change")

        print("="*70)
        sys.exit(1)

    print(f"Loaded Instructor's N (Modulus) with {INSTRUCTOR_N.bit_length()} bits.")
    print(f"Loaded Instructor's E (Exponent): {INSTRUCTOR_E}")
    print(f"Message to encrypt: '{MY_MESSAGE}'")

    # 2. Getting Bit Length
    # bitlen should be the length in bits of the RSA modulo N
    bitlen = INSTRUCTOR_N.bit_length()

    # 3. Encoding the Message
    # Encode the string s into a list of integers [cite: 20, 21]
    m_list = encodeText(MY_MESSAGE, bitlen)

    # For the warm-up, we expect a single integer
    if len(m_list) > 1:
        print("\nERROR: Your message is too long and was encoded into")
        print(f"       {len(m_list)} integers. Please use a shorter message for this test.")
        sys.exit(1)

    m = m_list[0]
    print(f"\nEncoded message (m): {m}")

    # 4. Encrypting the Message
    # The ciphertext is c = m^e mod N
    # We use Python's pow(base, exp, mod) for this
    print("Encrypting... (this may take a moment)")
    ciphertext = pow(m, INSTRUCTOR_E, INSTRUCTOR_N)

    # 5. Posting the Result
    print("\n" + "="*70)
    print(" RSA Encryption Successful!")
    print("\nMy ciphertext is:")
    print(ciphertext)
    print("\n" + "="*70)
    print("COPYing this entire number and I have PASTE it into the Google Sheet")



if __name__ == '__main__':
    main()

Starting Lab 03 - Part 1: RSA Warm-up
Loaded Instructor's N (Modulus) with 1023 bits.
Loaded Instructor's E (Exponent): 65537
Message to encrypt: 'Hello Instructor! This is Md Ismail Hossain.'

Encoded message (m): 1663907998830892081605551418917162906190535974899574738140659620776306442120385537834694056825538644567368
Encrypting... (this may take a moment)

 RSA Encryption Successful!

My ciphertext is:
4368808105426677886076940076648751732063580567336747290292067233197829964999147790799728305611538990937301706557108465488970365965806153633426887996373896969505328942189246512174169675740894278909641470871615534895241142427183534333929291252970314342845262907770126890707467735628595767654439916867326834775

COPYing this entire number and I have PASTE it into the Google Sheet


#### PART 2: RSA KEY GENERATION (YOUR KEY PAIR)

In [8]:
# --- Configuration ---
KEY_BITS = 1024
PRIME_BITS = KEY_BITS // 2
PUBLIC_EXPONENT = 65537

print(f"Generating {KEY_BITS}-bit RSA key pair...")
p = 0
q = 0

# Generate prime 'p'
print(f"Finding first prime p ({PRIME_BITS} bits)...")
start_time = time.time()
while not is_prime(p, 100):
    p = generate_prime_candidate(PRIME_BITS)
print(f"Found p {p} \nin {time.time() - start_time:.2f} seconds.")

# Generate prime 'q'
print(f"Finding second prime q ({PRIME_BITS} bits)...")
start_time = time.time()
while not is_prime(q, 100) or p == q:
    q = generate_prime_candidate(PRIME_BITS)
print(f"Found q in {time.time() - start_time:.2f} seconds.")

# Calculate N and phi(N)
N = p * q
phiN = (p - 1) * (q - 1)

# Set public exponent 'e' and check
e = PUBLIC_EXPONENT
(g, x, y) = egcd(e, phiN)



Generating 1024-bit RSA key pair...
Finding first prime p (512 bits)...
Found p 13401465205817003348601205063391223758003303456253861510900900273127168844483340297880692866534468984549102394177419121302238538042100663299900966646198333 
in 0.12 seconds.
Finding second prime q (512 bits)...
Found q in 0.40 seconds.


In [9]:
if g != 1:
    print(f"ERROR: e={e} is not relatively prime to phiN. This is rare. Re-run cell.")
else:
    # Calculate private exponent 'd'
    d = x % phiN

    print("\n" + "="*70)
    print(" RSA KEY GENERATION COMPLETE")
    print("="*70)

    print("\n--- PUBLIC KEY (Post this on the Google Sheet) ---")
    print("\nMY_N (Modulus):")
    print(N)
    print("\nMY_E (Public Exponent):")
    print(e)

    print("\n\n--- PRIVATE KEY (need to keep this SECRET!) ---")
    print("\nMY_D (Private Exponent):")
    print(d)


 RSA KEY GENERATION COMPLETE

--- PUBLIC KEY (Post this on the Google Sheet) ---

MY_N (Modulus):
146518581396870493989699713832787712967058776569111703686364211290154977756555265549041656786181188342907071817405588016629590046187250469660689163714479635347619276145588947425312239370685183973901836441042276331808925406940019991462805780289388263958628964106686761798837776632495891354890708359757857115289

MY_E (Public Exponent):
65537


--- PRIVATE KEY (need to keep this SECRET!) ---

MY_D (Private Exponent):
113169211137366440419283755957943055531875356972831140281119922723158597037350314205601243061423192302332361496514501203927397472237035854162138722663944915230854182439562956317113306741443535157538532514544226179624743415190289605446701879700643248415947365854236299924676246568443250212881483962759839979713


In [10]:
print("\n\n--- PRIMES (For my report) ---")
print("\nMY_P:")
print(p)
print("\nMY_Q:")
print(q)
print("\n" + "="*70)



--- PRIMES (For my report) ---

MY_P:
13401465205817003348601205063391223758003303456253861510900900273127168844483340297880692866534468984549102394177419121302238538042100663299900966646198333

MY_Q:
10933027034482247442598215141528375195950456088496037246048568336017398081674219692785216018833965425874585701426968347308622403813513381619946989716789133



#### PART 3: RSA DECRYPTION (INSTRUCTOR'S REPLY)

In [11]:
# need to Copy my key values from the output of the cell above.
MY_N = 104483561100600667109996160006670378950291734000878045733228912157531980809747598393552921224199418529564261886798219687643091435826581953946024055719996021071476156457716650610903296808596408433990573062175368874054966653908827915242194477904893194222559583463664458118844938151671595295515817741730746474437
MY_D = 6346782073355375677325704761990246404338181379335268627858832404582675673338803869642098042228632454433302204424122443451899644567582018686530078670389308001518011047190716390076298512162437892828468506802988317387277545949752676486390413218237832191448043629761769290762733477847233277921766343907136206721

# Getting the instructor's reply (ciphertext) from the Google Sheet
#INSTRUCTOR_REPLY_C = need to ASK to the Professor or other group.

#### Part 4a - Diffie-Hellman & AES Encryption

This cell generates our DH key, computes a shared AES key, and encrypts a message. we will Post our DH PUBLIC KEY to the sheet. Then, we will get another group's DH public key (which is 'pkDH' in the sheet) and paste it in.

In [17]:
# Part 4a - DH Key Exchange & AES Encryption

print("Starting Part 4: Diffie-Hellman and AES")

#Generating our DH key pair
print("Generating our DH key pair...")
# Creating a private key x in range [1, q-1]
dh_x = (int.from_bytes(os.urandom(256), byteorder='little') % (qDH - 1)) + 1 # this is my private key
#
# Computing public key pk = g^x mod p
my_pkDH = pow(gDH, dh_x, pDH)   # we can share this public key

print("\n--- OUR DH PUBLIC KEY (Posted this on the Google Sheet) ")
print("\nMY_DH_PK:")
print(my_pkDH)
print("\n(Keeping our 'dh_x' (private key) value SECRET!)")


# Getting professor's DH public key from the sheet
PK_prof = 231496621204370508895931748857755688773955557370722566546524279241681783924615459222281328781846958709671178933550268979137824811356380142723458582689238338881602937866599482596517635627201490862987705580528580122030574292913825460656414418530301010266004326589419172467364887958500892750202983947301330827393531207626744229962720413918512823808690023548441036635272327523845223428985955360366428904878223170012768633135103661638106507748637163377957664799771569021118234716556142527020139132381453117289393367071863842364636961374713484929024198420198291374856545022707013121483896955505081060905509151197548949686

#  Message to send via AES
AES_MESSAGE = "This is a secret message encrypted with AES-CTR!"
# -------------------------------

if PK_prof:
    # Computing shared key k = (pk_2)^x mod p
    print("\nComputing shared secret...")
    k = pow(PK_prof, dh_x, pDH)

    # Deriving 128-bit (16-byte) AES key from k

    shared_key_aes = (k & ((1 << 128) - 1)).to_bytes(16, byteorder='little')
    print(f"Derived 16-byte AES key.")

    # Encrypting with AES-CTR
    print(f"Encrypting message: '{AES_MESSAGE}'")
    plaintext = AES_MESSAGE.encode('utf-8')
    (iv, ciphertext) = encryptAESCTR(shared_key_aes, plaintext)

    # Encoding for Google Sheet using Base64
    iv_string = base64.b64encode(iv).decode('utf-8')

    ciphertext_string = base64.b64encode(ciphertext).decode('utf-8')

    print("\n" + "="*70)
    print(" AES Encryption Successful!")
    print("\nneed to Post these two (dynamic) strings on the Google Sheet:")
    print(f"\nIV (Base64):\n{iv_string}")
    print(f"\nCiphertext (Base64):\n{ciphertext_string}")
    print("="*70)
else:
    print("\n" + "❌ ERROR: Please fill in OTHER_GROUP_PK")
    print("in the cell above before running this one.")

Starting Part 4: Diffie-Hellman and AES
Generating our DH key pair...

--- OUR DH PUBLIC KEY (Posted this on the Google Sheet) 

MY_DH_PK:
1192450075182419528513166702436200630714213647544307683871425137706742207445441128877752364755749116051520604375991659847210391884691113969004965660936565589140302368573246981483833747814534765553219837951317145459735719532719954199060728974182425962426986012186337289291436981505328575962215984523495760237963116987596876291672188362736283138624789079173232941475147560691915056449455412180265008804610419673459845262364139170269208453435305420833462483623184227108084505769106930588145042743961359958066663453301312936310749343252452388777064326360813493903382461035927769296835566882752944686651676961837674218183

(Keeping our 'dh_x' (private key) value SECRET!)

Computing shared secret...
Derived 16-byte AES key.
Encrypting message: 'This is a secret message encrypted with AES-CTR!'

 AES Encryption Successful!

need to Post these two (dynamic) strings

!!! actually it’s a collaborative lab activity. all we need to sit with another group, so that we can share our initialization Vector (IV), and can finish the lab successfully. It will require no more than 30 mins.

also, we need to ask the other group what was the professor’s reply for the RSA Decryption part.

